In [5]:
import pandas as pd 
import pycountry
from util import *

In [6]:
df = load_dataset()
countries = get_country_dict(df)
local_store_df = get_local_stores(df, country_code="AT")

In [7]:
local_store_df

,name,url,street_address,city,zip_code,country_code,country,open_hours,latitude,longitude
0,Neubaugasse,https://www.starbucks.com/store-locator/store/161,Neubaugasse 4,Wien,1070,AT,Austria,"Wednesday : 7:00 AM to 9:00 PM, Thursday : 7:0...",48.198655,16.350052
1,Mariahilfer Strasse,https://www.starbucks.com/store-locator/store/160,Mariahilfer Strasse 23-25,Wien,1060,AT,Austria,"Wednesday : 7:30 AM to 10:00 PM, Thursday : 7:...",48.200706,16.358635
2,Millennium City,https://www.starbucks.com/store-locator/store/164,Handelskai 94-96,Wien,1200,AT,Austria,"Wednesday : 7:00 AM to 9:00 PM, Thursday : 7:0...",48.240344,16.386985
3,AKH Allgemeines Krankenhaus,https://www.starbucks.com/store-locator/store/171,Waehringer Guertel 18-20,Wien,1090,AT,Austria,"Wednesday : 6:30 AM to 7:30 PM, Thursday : 6:3...",48.220408,16.347473
4,Rotenturmstrasse,https://www.starbucks.com/store-locator/store/169,Rotenturmstrasse 22,Wien,1010,AT,Austria,"Wednesday : 7:00 AM to 9:30 PM, Thursday : 7:0...",48.211299,16.376093
5,Invalidenstrasse,https://www.starbucks.com/store-locator/store/172,Invalidenstrasse 11,Wien,1030,AT,Austria,"Wednesday : 6:30 AM to 9:00 PM, Thursday : 6:3...",48.205817,16.385848
6,Michaelerplatz,https://www.starbucks.com/store-locator/store/170,Reitschulgasse 4,Wien,1010,AT,Austria,"Wednesday : 8:00 AM to 8:00 PM, Thursday : 8:0...",48.207627,16.367094
7,Karlsplatzpassage,https://www.starbucks.com/store-locator/store/168,Karlsplatz Hauptpassage,Wien,1010,AT,Austria,"Wednesday : 6:30 AM to 8:00 PM, Thursday : 6:3...",48.200786,16.368764
8,Donauzentrum,https://www.starbucks.com/store-locator/store/...,Wagramer Strasse 81,Wien,1220,AT,Austria,"Wednesday : 7:30 AM to 8:30 PM, Thursday : 7:3...",48.243246,16.434104


# Starts Here:

In [8]:
# O(n)
def getCoordinates(store_df):
  n = len(store_df)

  coordinates = []
  for i in range(n):
    latitude = store_df["latitude"][i]
    longitude = store_df["longitude"][i]

    coordinates.append((latitude, longitude))

  return coordinates

In [9]:
from geopy import distance

def getDistance(coord1, coord2):
  return distance.distance(coord1, coord2).km

In [10]:
# O(n)
def getTotalDistance(centerIndex, coordinates, distance_func):
  totalDistance = 0

  for i in range(len(coordinates)):
    if i != centerIndex:
      totalDistance += distance_func(coordinates[centerIndex], coordinates[i])*2
  
  return totalDistance

In [11]:
local_store_coordinates = getCoordinates(local_store_df)
local_store_coordinates

[(48.198655, 16.350052),
 (48.200706, 16.358635),
 (48.240344, 16.386985),
 (48.220408, 16.347473),
 (48.211299, 16.376093),
 (48.205817, 16.385848),
 (48.207627, 16.367094),
 (48.200786, 16.368764),
 (48.243246, 16.434104)]

## 1. Brute force: O(n2 + n) = O(n2)

In [12]:
def selectCenter_1(coordinates, distance_func):
  n = len(coordinates)

  distances = [[0 for _ in range(n)] for _ in range(n)]
  for i in range(n):
    for j in range(i+1, n):
      distance = distance_func(coordinates[i], coordinates[j])
      distances[i][j] = distances[j][i] = distance
  
  minIndex = -1
  minDistance = -1
  for i in range(n):
    totalDistance = sum(distances[i])
    if totalDistance < minDistance or minIndex < 0:
      minDistance = totalDistance
      minIndex = i

  return minIndex

In [13]:
centerIndex1 = selectCenter_1(local_store_coordinates, getDistance)
centerIndex1

6

In [14]:
getTotalDistance(centerIndex1, local_store_coordinates, getDistance)

35.78768525182979

# 2. O(2n) = O(n)

In [15]:
def selectCenter_2(coordinates, distance_func):
  n = len(coordinates)

  center = [0, 0]
  for coordinate in coordinates:
    center[0] += coordinate[0]
    center[1] += coordinate[1]
  center[0] /= n
  center[1] /= n
  
  minDiff = -1
  minIndex = -1
  for i in range(n):
    diff = distance_func(center, coordinates[i])
    if diff < minDiff or minIndex < 0:
      minDiff = diff
      minIndex = i
  
  return minIndex

In [16]:
centerIndex2 = selectCenter_2(local_store_coordinates, getDistance)
centerIndex2

4

In [17]:
getTotalDistance(centerIndex2, local_store_coordinates, getDistance)

36.87171302202011

# 3. O(n log n + 2n) = O(n log n)

In [46]:
def selectCenter_3(coordinates, distance_func):
  n = len(coordinates)

  hypotenuses = []
  for i in range(n):
    hypotenuse = (coordinates[i][0]**2 + coordinates[i][1]**2)**(1/2)
    hypotenuses.append(hypotenuse)
  
  sorted_hypotenuses = sorted(hypotenuses)  # Sorting (We may implement this ourselves)
  
  left = sorted_hypotenuses[0]
  right = sorted_hypotenuses[n-1]
  minDistance = right - left
  minHypotenus = -1
  for i in range(1, n):
    distance = max(sorted_hypotenuses[i] - left, right - sorted_hypotenuses[i])
    if distance < minDistance:
      minDistance = distance
      minHypotenus = sorted_hypotenuses[i]

  return hypotenuses.index(minHypotenus)

In [47]:
centerIndex3 = selectCenter_3(local_store_coordinates, getDistance)
centerIndex3

4

In [48]:
getTotalDistance(centerIndex3, local_store_coordinates, getDistance)

36.87171302202011